# Flight Data Cleaning, EDA , Feature Engineering and Predictions

#### Importing Basic Liabraries for EDA and Feature Engineering

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
init_notebook_mode(connected=True)

cf.go_offline()
%matplotlib inline

#### Let's Import Train and Test data and combine them to ensure we train same set of data to algorithm. Because post handling Nan Values and feature engineering, same data shoul be splitted into training and testing set to get predictions on test data.

In [2]:
df_train = pd.read_excel('Data_Train.xlsx')

In [3]:
df_test = pd.read_excel('Test_set.xlsx')

In [4]:
final_df = df_train.append(df_test)

C:\Users\Krishna Kanhaiya\AppData\Local\Temp\ipykernel_11368\585889476.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Let's see once how merged dataframe look like and what all features we got :

In [5]:
final_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [6]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


##### We can see here, there are multiple columns which can be transformed. So, Let's start with Tranformation and feature Engineering

### Transforming Date of Journey Column

In [7]:
final_df['Date']=final_df['Date_of_Journey'].apply(lambda str:str.split('/')[0])
final_df['Month']=final_df['Date_of_Journey'].apply(lambda str:str.split('/')[1])
final_df['Year']=final_df['Date_of_Journey'].apply(lambda str:str.split('/')[2])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
 11  Date             13354 non-null  object 
 12  Month            13354 non-null  object 
 13  Year             13354 non-null  object 
dtypes: float64(1), object(13)
memory usage: 1.5+ MB


We have transformed the Date of Journey Column into Date, month and year but it's datatype is object. So, Let's convert it into integers

In [8]:
final_df['Date'] = final_df['Date'].astype(int)
final_df['Month'] = final_df['Month'].astype(int)
final_df['Year'] = final_df['Year'].astype(int)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
 11  Date             13354 non-null  int32  
 12  Month            13354 non-null  int32  
 13  Year             13354 non-null  int32  
dtypes: float64(1), int32(3), object(10)
memory usage: 1.4+ MB


In [9]:
#Dropping original column Date as we have transformed the same.

final_df.drop('Date_of_Journey',axis=1,inplace=True)

### Transforming Arrival Time

Arrival Time Can be tranformed into Arrival Hours and Arrival Minutes

In [10]:
final_df['Arrival_hour'] = final_df['Arrival_Time'].str.split(':').str[0]
final_df['Arrival_min'] = final_df['Arrival_Time'].str.split(':').str[1].str.split(' ').str[0]

In [11]:
final_df['Arrival_hour'] = final_df['Arrival_hour'].astype(int)
final_df['Arrival_min'] = final_df['Arrival_min'].astype(int)

In [12]:
final_df.drop('Arrival_Time',axis=1,inplace=True)

In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Duration         13354 non-null  object 
 6   Total_Stops      13353 non-null  object 
 7   Additional_Info  13354 non-null  object 
 8   Price            10683 non-null  float64
 9   Date             13354 non-null  int32  
 10  Month            13354 non-null  int32  
 11  Year             13354 non-null  int32  
 12  Arrival_hour     13354 non-null  int32  
 13  Arrival_min      13354 non-null  int32  
dtypes: float64(1), int32(5), object(8)
memory usage: 1.3+ MB


### Transforming Departure Time

Departure Time Can be tranformed into Departure Hours and Departure Minutes

In [14]:
final_df['Dep_hour']=final_df['Dep_Time'].apply(lambda x:x.split(':')[0])
final_df['Dep_min']=final_df['Dep_Time'].apply(lambda x:x.split(':')[1])

final_df['Dep_hour']=final_df['Dep_hour'].astype(int)
final_df['Dep_min']=final_df['Dep_min'].astype(int)

final_df.drop('Dep_Time',axis=1,inplace=True)

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Duration         13354 non-null  object 
 5   Total_Stops      13353 non-null  object 
 6   Additional_Info  13354 non-null  object 
 7   Price            10683 non-null  float64
 8   Date             13354 non-null  int32  
 9   Month            13354 non-null  int32  
 10  Year             13354 non-null  int32  
 11  Arrival_hour     13354 non-null  int32  
 12  Arrival_min      13354 non-null  int32  
 13  Dep_hour         13354 non-null  int32  
 14  Dep_min          13354 non-null  int32  
dtypes: float64(1), int32(7), object(7)
memory usage: 1.3+ MB


### Feature Engineering on 'Total Stops'

It can be seen in the data that Total Stops can be used as categorical variable. Let's map that accordingly

In [15]:
final_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [16]:
#Mapping Above Unique Values as Categories in the Total Stops

final_df['Total_Stops'] = final_df['Total_Stops'].map({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4,'NaN':1})

final_df['Total_Stops'] = final_df['Total_Stops'].fillna(0)
final_df['Total_Stops'] = final_df['Total_Stops'].astype(int)

In [17]:
#Dropping Original Column
final_df.drop('Route',axis=1,inplace=True)

In [18]:
final_df

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,2h 50m,0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1,No info,13302.0,1,3,2019,21,35,16,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,Air India,Kolkata,Banglore,23h 55m,1,No info,NaN,6,6,2019,20,25,20,30
2667,IndiGo,Kolkata,Banglore,2h 35m,0,No info,NaN,27,3,2019,16,55,14,20
2668,Jet Airways,Delhi,Cochin,6h 35m,1,No info,NaN,6,3,2019,4,25,21,50
2669,Air India,Delhi,Cochin,15h 15m,1,No info,NaN,6,3,2019,19,15,4,0


### Feature Engineering on 'Duration'

We will now try to calculate total duration in minutes so that data can be in single unit to analyze properly

In [19]:
duration_hour = final_df['Duration'].str.split(' ').str[0].str.split('h').str[0]

In [20]:
#duration_hour.astype(int) #Convert to int for addition with duration mins

#This will lead to error telling there is inconsistent  value = 5m (As we have splitted by 'h' and encountered 'm'). Let's check where are these records

In [21]:
duration_hour[duration_hour == '5m']

6474    5m
2660    5m
Name: Duration, dtype: object

In [22]:
#Removing these entries as it's incorrect (duration of flying can't be 5min only)
final_df.drop(6474,axis=0,inplace=True)
final_df.drop(2660,axis=0,inplace=True)

In [23]:
duration_hour = final_df['Duration'].str.split(' ').str[0].str.split('h').str[0].astype(int)
duration_mins = final_df['Duration'].str.split(' ').str[1].str.split('m').str[0].fillna(0).astype(int)

final_df['Duration_in_mins'] = (duration_hour*60)+duration_mins  #Calculating Total Duration in Mins

final_df.drop('Duration',axis=1,inplace=True)  #Dropping original Column

In [24]:
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_in_mins
0,IndiGo,Banglore,New Delhi,0,No info,3897.0,24,3,2019,1,10,22,20,170
1,Air India,Kolkata,Banglore,2,No info,7662.0,1,5,2019,13,15,5,50,445
2,Jet Airways,Delhi,Cochin,2,No info,13882.0,9,6,2019,4,25,9,25,1140
3,IndiGo,Kolkata,Banglore,1,No info,6218.0,12,5,2019,23,30,18,5,325
4,IndiGo,Banglore,New Delhi,1,No info,13302.0,1,3,2019,21,35,16,50,285


In [25]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Airline           13351 non-null  object 
 1   Source            13351 non-null  object 
 2   Destination       13351 non-null  object 
 3   Total_Stops       13351 non-null  int32  
 4   Additional_Info   13351 non-null  object 
 5   Price             10681 non-null  float64
 6   Date              13351 non-null  int32  
 7   Month             13351 non-null  int32  
 8   Year              13351 non-null  int32  
 9   Arrival_hour      13351 non-null  int32  
 10  Arrival_min       13351 non-null  int32  
 11  Dep_hour          13351 non-null  int32  
 12  Dep_min           13351 non-null  int32  
 13  Duration_in_mins  13351 non-null  int32  
dtypes: float64(1), int32(9), object(4)
memory usage: 1.1+ MB


We are now left with 4 Features : Airline , Source,  Destination &  Additional_Info 
Let's inspect one of the feature 

In [26]:
final_df['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [27]:
fig = px.box(final_df, x='Airline', y='Price')
fig.update_layout(template='plotly_dark', title='Box Plots showing Prices 5 Number Summnary of Airlines',
                 font = dict(family = "PT Sans", size = 14))

fig

#sns.catplot(x ='Airline',y='Price', data = df_train.sort_values('Price', ascending = False), kind='boxen', height = 6, aspect = 3)

## Label Encoding

In [28]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

In [29]:
final_df['Airline']=labelencoder.fit_transform(final_df['Airline'])
final_df['Source']=labelencoder.fit_transform(final_df['Source'])
final_df['Destination']=labelencoder.fit_transform(final_df['Destination'])
final_df['Additional_Info']=labelencoder.fit_transform(final_df['Additional_Info'])

In [30]:
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_in_mins
0,3,0,5,0,8,3897.0,24,3,2019,1,10,22,20,170
1,1,3,0,2,8,7662.0,1,5,2019,13,15,5,50,445
2,4,2,1,2,8,13882.0,9,6,2019,4,25,9,25,1140
3,3,3,0,1,8,6218.0,12,5,2019,23,30,18,5,325
4,3,0,5,1,8,13302.0,1,3,2019,21,35,16,50,285


As labelled encoded data is Nominal Categorical data, so we will perform OneHotEncoding

In [31]:
final_df= pd.get_dummies(final_df, columns=['Airline','Source','Destination','Additional_Info'], drop_first=True)

In [32]:
final_df.columns

Index(['Total_Stops', 'Price', 'Date', 'Month', 'Year', 'Arrival_hour',
       'Arrival_min', 'Dep_hour', 'Dep_min', 'Duration_in_mins', 'Airline_1',
       'Airline_2', 'Airline_3', 'Airline_4', 'Airline_5', 'Airline_6',
       'Airline_7', 'Airline_8', 'Airline_9', 'Airline_10', 'Airline_11',
       'Source_1', 'Source_2', 'Source_3', 'Source_4', 'Destination_1',
       'Destination_2', 'Destination_3', 'Destination_4', 'Destination_5',
       'Additional_Info_1', 'Additional_Info_2', 'Additional_Info_3',
       'Additional_Info_4', 'Additional_Info_5', 'Additional_Info_6',
       'Additional_Info_7', 'Additional_Info_8', 'Additional_Info_9'],
      dtype='object')

##### Let's see how our final transformed data look like :

In [33]:
final_df.shape[1]

39

In [34]:
final_df.head()

,Total_Stops,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_in_mins,...,Destination_5,Additional_Info_1,Additional_Info_2,Additional_Info_3,Additional_Info_4,Additional_Info_5,Additional_Info_6,Additional_Info_7,Additional_Info_8,Additional_Info_9
0,0,3897.0,24,3,2019,1,10,22,20,170,...,1,0,0,0,0,0,0,0,1,0
1,2,7662.0,1,5,2019,13,15,5,50,445,...,0,0,0,0,0,0,0,0,1,0
2,2,13882.0,9,6,2019,4,25,9,25,1140,...,0,0,0,0,0,0,0,0,1,0
3,1,6218.0,12,5,2019,23,30,18,5,325,...,0,0,0,0,0,0,0,0,1,0
4,1,13302.0,1,3,2019,21,35,16,50,285,...,1,0,0,0,0,0,0,0,1,0


##### We will now seprate the training and testing dataframes as we have to predict the prices for test data

In [35]:
df_test = final_df[final_df['Price'].isnull()]
df_train = final_df[~final_df['Price'].isnull()]

##### Let's now Split Training data into dependent(y) and independent(X) features

In [94]:
X = df_train.drop('Price',axis=1)
y = df_train['Price']

##### Apply train test split on the X and y

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

### Applying XGB Regressor

In [96]:
from xgboost import XGBRegressor
model =  XGBRegressor()
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [107]:
y_pred =  model.predict(X_test)
print('Training Score :',model.score(X_train, y_train))
print('Test Score     :',model.score(X_test, y_test))

Training Score : 0.970333249551306
Test Score     : 0.8933387692257291


In [118]:
temp_df = pd.DataFrame({'y_test':y_test,'y_pred':y_pred})
temp_df

,y_test,y_pred
0,5989.0,4470.192383
1,11982.0,12098.504883
2,10975.0,10518.837891
3,10783.0,14706.885742
4,8098.0,7700.408691
...,...,...
3200,7878.0,7682.495117
3201,4423.0,4162.086426
3202,6795.0,7175.380859
3203,12373.0,12334.022461


### Let's Plot our y_test and y_pred values to see how our model performed

In [128]:
pd.set_option("plotting.backend", "plotly")
temp_df[:50].plot(kind='line')

#### It is clearly seen that our model is behaving pretty good on the data as both points have kind of same graph.

### Now Let's calculate test dataframe

In [82]:
df_test.head()

,Total_Stops,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_in_mins,...,Destination_5,Additional_Info_1,Additional_Info_2,Additional_Info_3,Additional_Info_4,Additional_Info_5,Additional_Info_6,Additional_Info_7,Additional_Info_8,Additional_Info_9
0,1,NaN,6,6,2019,4,25,17,30,655,...,0,0,0,0,0,0,0,0,1,0
1,1,NaN,12,5,2019,10,20,6,20,240,...,0,0,0,0,0,0,0,0,1,0
2,1,NaN,21,5,2019,19,0,19,15,1425,...,0,0,0,0,0,1,0,0,0,0
3,1,NaN,21,5,2019,21,0,8,0,780,...,0,0,0,0,0,0,0,0,1,0
4,0,NaN,24,6,2019,2,45,23,55,170,...,0,0,0,0,0,0,0,0,1,0


In [83]:
X = df_test.drop('Price',axis=1)

In [84]:
pred = model.predict(X)

In [91]:
df_test['Price']=pred

#pd.DataFrame(pred,columns=['Prediction'])

In [92]:
df_test

,Total_Stops,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_in_mins,...,Destination_5,Additional_Info_1,Additional_Info_2,Additional_Info_3,Additional_Info_4,Additional_Info_5,Additional_Info_6,Additional_Info_7,Additional_Info_8,Additional_Info_9
0,1,14365.401367,6,6,2019,4,25,17,30,655,...,0,0,0,0,0,0,0,0,1,0
1,1,4669.960449,12,5,2019,10,20,6,20,240,...,0,0,0,0,0,0,0,0,1,0
2,1,12759.782227,21,5,2019,19,0,19,15,1425,...,0,0,0,0,0,1,0,0,0,0
3,1,10794.085938,21,5,2019,21,0,8,0,780,...,0,0,0,0,0,0,0,0,1,0
4,0,4137.477539,24,6,2019,2,45,23,55,170,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,9802.762695,6,6,2019,20,25,20,30,1435,...,0,0,0,0,0,0,0,0,1,0
2667,0,5319.214844,27,3,2019,16,55,14,20,155,...,0,0,0,0,0,0,0,0,1,0
2668,1,16903.822266,6,3,2019,4,25,21,50,395,...,0,0,0,0,0,0,0,0,1,0
2669,1,12612.032227,6,3,2019,19,15,4,0,915,...,0,0,0,0,0,0,0,0,1,0
